In [1]:
from fomo.pipelines.train import Learner
from fomo.pipelines.types.learner_args import LearnerArgs
from fomo.utils.data.datasets import DatasetInitializer, PROMPTS
import torch

/home/jupyter/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NUM_OF_SAMPLES = 16

In [3]:
#TRAIN_SUBSAMPLES = ["base", "all"]
#TRAIN_SUBSAMPLES = ["base"]
TRAIN_SUBSAMPLES = ["base"]
MODEL_TYPE = "clip_transformer_adapter_text"


In [4]:
#DATASETS = ["cifar10", "caltech101", "oxford_pets", "oxford_flowers", "food101", "stanford_cars", "fgvc_aircraft", "dtd", "sun397"]
#DATASETS = ["caltech101", "oxford_pets", "oxford_flowers", "food101", "stanford_cars", "fgvc_aircraft", "dtd", "sun397"]
DATASETS = ["caltech101", "oxford_pets", "oxford_flowers", "fgvc_aircraft"]
#DATASETS = ["cifar10"]

In [5]:
for train_subsample in TRAIN_SUBSAMPLES:
    for dataset_name in DATASETS:
        print(f"Training {MODEL_TYPE} on {dataset_name} on {train_subsample} subsample.")
        
        _dataset = DatasetInitializer.from_str(dataset_name).value(train=True)
        labels = _dataset.labels
        prompt_template = PROMPTS[dataset_name]
        num_of_classes = len(labels) // 2 if train_subsample == "base" else len(labels)
        #num_of_classes = len(labels) - (len(labels) // 5) if train_subsample == "base" else len(labels)
        train_eval_size = [NUM_OF_SAMPLES * num_of_classes, NUM_OF_SAMPLES * num_of_classes]

        learner_args = LearnerArgs(model_type=MODEL_TYPE, dataset=dataset_name)
        learner_args.device = "cuda"
        learner_args.epochs = 200
        learner_args.model_type = MODEL_TYPE
        learner_args.train_eval_size = train_eval_size
        learner_args.text_prompt_template = prompt_template
        learner_args.print_freq = 200
        learner_args.save_freq = 200
        learner_args.learning_rate = 5e-3
        learner_args.use_wandb = False
        learner_args.patience = 5
        learner_args.info = "downsampling 256"
        learner_args.batch_size = 256
        learner_args.momentum = 0.5
        learner_args.train_subsample = train_subsample
        learner_args.dataset = dataset_name

        learner = Learner(learner_args)
        learner.run()

Training clip_transformer_adapter_text on caltech101 on base subsample.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Turning off gradients in both the image and the text encoder
Parameters to be updated: {'adapter.mha.in_proj_bias', 'adapter.mha.in_proj_weight', 'adapter.upsampler.0.weight', 'adapter.downsampler.0.weight', 'adapter.upsampler.2.weight', 'adapter.downsampler.0.bias', 'adapter.downsampler.1.weight', 'adapter.mha.out_proj.weight', 'adapter.upsampler.0.bias', 'adapter.downsampler.1.bias', 'adapter.upsampler.2.bias', 'adapter.mha.out_proj.bias'}
Number of learnable paramms: 313152


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch: [0][0/4]	Time  3.582 ( 3.582)	Data  2.188 ( 2.188)	Loss 2.3745e-01 (2.3745e-01)	Acc@1  95.31 ( 95.31)


 25%|██▌       | 1/4 [00:02<00:06,  2.15s/it]

Validate: [0/4]	Time  2.147 ( 2.147)	Loss 9.4687e-02 (9.4687e-02)	Prompt Acc@1  97.27 ( 97.27)


...

 * Prompt Acc@1 29.592


 14%|█▍        | 1/7 [00:08<00:52,  8.76s/it]

Validate: [0/7]	Time  8.764 ( 8.764)	Loss 2.0410e+00 (2.0410e+00)	Prompt Acc@1  39.45 ( 39.45)


100%|██████████| 7/7 [00:16<00:00,  2.33s/it]

 * Prompt Acc@1 35.333
